Can I also just use the tabular approach? likely...

The big difference here, is that now the state changes. The right action to take depends on what state you are in!

Its like you are walking through various rooms in the casino. For now we can assume that its a one step problem, and then next state is random.

Why doesn't the tabular case end up working in reality?....

Well for the action space... I don't think you can do much better actually than a average reward + epsiolon greedy exploration/explotation strategy.


Ok here is a fantastic post: https://lilianweng.github.io/posts/2018-01-23-multi-armed-bandit/#summary

Some options:
1. No Exploration: Greedy algorithim, we always do what we think is best
2. Random Exploration: E greedy algorithim, we randomly try new actions
3. Smart exploration: Upper Confidence bounds (UCB), we select actions to try that we are uncertain about.

For multi-step problems definetly a lot more complicated actually. I really like building the intutions from the ground up though.


How many states???

lets say we have 10 states. ok then I can still just solve with tabular. I like the idea of the state being a QR code... :) its like the first step towards an image. It can still just be a key look up... 100 states... 

Next is than a grey scale image, something mabye like mnist? 

Ok lets start with just a simple 10x10 binary image. How many states are there? 10x10 = 100, 2^100 ~ 10^30 states... LOL... ok 

Ok I will use the same approach. I will flatten the image, and then turn it into a key for a dictionary. that will access the table, and then increment that table... same thing,
I can display the various bar charts, randomly select some. Its just going to take longer to fill in all the values.

---
The only reason you are able to do better is that there is structure.
If each state is truley random, then we are hoped...
- If you change the color of the cup, the same actions should still apply
- Wether its light or dark, the same actions should still apply, etc. etc..

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import bam_gym # import to that __init__.py is run to register envs
from bam_gym import print_reset, print_step

In [ ]:

# Square numbers are easy to see as square
n_arms = 5
n_states = 10
env = gym.make("bam_local/ContextBandit", render_mode="human", clip=10, n_arms=n_arms, n_states=n_states)
print("Action Space:", env.action_space)
print("Observation Space:", env.observation_space)
print("Reset:", env.reset())

env.unwrapped.render(sample_states=2)

In [ ]:
state, info = env.reset()
print_reset(state, info)

def reset_env(env, terminated, truncated, info) -> bool:
    if terminated or truncated:
        if env.metadata['autoreset_mode'] != gym.vector.AutoresetMode.SAME_STEP:
            return True

    return False

for i in range(10):
    action = env.action_space.sample()
    # action = policy(state)
    state, reward, terminated, truncated, info = env.step(action)
    print_step(action, state, reward, terminated, truncated, info, i)

    if reset_env(env, terminated, truncated, info):
        state, info = env.reset()

env.close()


In [ ]:
# I can use a super duper simple policy to beat this, essentially just average the rewards to date, and then pick the highest action with epislon greedy probability
# If you want to get a very accuarte understanding of the world, then you can continue to do random exploration, but to actually maximise reward you want to exploit as well!

class StateAveragePolicy():
    def __init__(self, n_actions, epsilon=0.2):

        self.state_action_table = {}
        self.n_actions = n_actions
        self.epsilon = epsilon
        self.iter = 0


    def make_new_action_table(self, state):
        self.state_action_table[state] = dict()
        self.state_action_table[state]['attempts'] = np.ones(self.n_actions)
        self.state_action_table[state]['reward'] = np.ones(self.n_actions)/self.n_actions



    def __call__(self, state):
        self.iter += 1

        if state not in self.state_action_table:
            self.make_new_action_table(state)

        action_values = self.state_action_table[state]['reward'] / self.state_action_table[state]['attempts']

        if np.random.random() <= self.epsilon:
            return np.random.randint(self.n_actions)
        else:
            return np.argmax(action_values)

    def update(self, state, action, reward):
        if state not in self.state_action_table:
            self.make_new_action_table(state)

        self.state_action_table[state]['attempts'][action] += 1
        self.state_action_table[state]['reward'][action] += reward
    
    @property
    def W(self):
        W = dict()
        for state in self.state_action_table:
            W[state] = self.state_action_table[state]['reward'] / self.state_action_table[state]['attempts']
        return W

    



policy = StateAveragePolicy(n_actions=n_arms, epsilon=0.2)
print(policy.W)

In [ ]:
reward_history = []
epsilon_history = []

In [ ]:
state, info = env.reset()
policy.epsilon = 0.0 # Manually tune epsilon 

for i in range(1000):
    action = policy(state)
    new_state, reward, terminated, truncated, info = env.step(action)
    print(new_state)
    # print_step(action, state, reward, terminated, truncated, info, i)
    policy.update(state, action, reward)
    reward_history.append(reward)
    epsilon_history.append(policy.epsilon)
    state = new_state

    if reset_env(env, terminated, truncated, info):
        state, info = env.reset()

env.close()
print(policy.W)

In [ ]:


for s in range(max(n_states, 5)):
    width = 0.4
    x = np.arange(n_arms)

    plt.figure(figsize=(10, 6))

    plt.bar(x - width/2, env.unwrapped.arm_thresholds[s], width, alpha=0.7, label="Actual")
    plt.bar(x + width/2, policy.W[s], width, alpha=0.7, label="Estimated")

    plt.ylim(0, 1)

    plt.xlabel('Arm')
    plt.ylabel('Threshold')
    plt.title(f'State {s}: Thresholds')
    plt.legend()

    plt.grid(True)
    plt.show()

In [ ]:
plt.plot(reward_history, label="Reward")
plt.plot(epsilon_history, label="Epsilon", color='red')
# Compute moving average
window_size = 50
if len(reward_history) >= window_size:
    moving_avg = np.convolve(reward_history, np.ones(window_size)/window_size, mode='valid')
    plt.plot(range(window_size-1, len(reward_history)), moving_avg, label=f"Moving Avg ({window_size})", color='orange')

plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("Reward History and Moving Average")
plt.legend()
plt.show()